Going to use pandas to read the excel files

In [18]:
# %pip install pandas
# %pip install rapidfuzz
# %pip install collections
# %pip install openpyxl

In [19]:
import pandas as pd
from rapidfuzz import process, fuzz
from collections import defaultdict
from openpyxl import load_workbook
from datetime import datetime, timedelta
import re
from dateutil.parser import parse

Read the files

In [20]:
students_df = pd.read_excel('/Users/edwardbrady/Library/CloudStorage/OneDrive-ImperialCollegeLondon/UROP-Exam-timetabling/input data/student list DONOT SORT ONLY FILTER.xlsx',header=None)
leaders_df = pd.read_excel('/Users/edwardbrady/Library/CloudStorage/OneDrive-ImperialCollegeLondon/UROP-Exam-timetabling/input data/Mech Eng module list 2025-26v2.xlsx',sheet_name=1,header=1)
# Load workbook and worksheet for useful dates
wb = load_workbook("/Users/edwardbrady/Library/CloudStorage/OneDrive-ImperialCollegeLondon/UROP-Exam-timetabling/input data/2025-26 Useful Dates.xlsx")
ws = wb.active

/Users/edwardbrady/Library/Python/3.12/lib/python/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Creating a list of exam names, from excel can see they're in Row 'A' which is 0 and start from column 'J' which is 9

In [21]:
exams = students_df.iloc[0, 9:].dropna().tolist()
print(exams)


['BUSI60039 Business Strategy', 'BUSI60046 Project Management', 'BUSI60040/BUSI60043 Corporate Finance Online/Finance & Financial Management', 'MECH70022 Advanced Control', 'MECH70045 Advanced Numerical Methods for Engineers  ', 'MECH70019 Advanced Stress Analysis', 'MECH70021Aircraft Engine Technology', 'ME ELEC70066 Applied Advanced Optimisation', 'MECH70016 Applied Vibration Engineering', 'MECH60024 Automotive Design with Motorsport', 'MECH70020 Combustion, Safety and Fire Dynamics', 'MECH60021/MECH70018 Computational Continuum Mechanics A/B', 'MECH70015 Computational Fluid Dynamics', 'MECH70026 Energy Systems', 'MECH70027 Environmental and Applied Fluid Dynamics', 'MECH60007/MECH70012 Finite Element Analysis and Applications A/B', 'MECH60006 Fluid Mechanics 3AB', 'MECH60002/MECH70005 Fracture Mechanics A/B', 'MECH70003 Future Clean Transport Technology', 'BIOE70016 Human Neuromechanical Control and Learning', 'MECH70009 Interfacing and Data Processing', 'MECH60004/MECH70042 Introdu

To calculate the number of days for the exam period we can assume it will be 3 weeks, weeks 31-33 and therefore it will be 21 days starting on monday of week 31

In [22]:
num_days = 21

To create a dictionary of students and there exams we will flick through each row, which is a studnet, and for every module that has an x , A or B add it to their exams. At the end add PEN as everyone does PEN


In [23]:
# Get the range of rows containing student data (from row 3 onward)
student_rows = students_df.iloc[2:, :]  # row index 3 and onward

# Initialize the dictionary
student_exams = {}

for _, row in student_rows.iterrows():
        cid = row[0]  # Column A = student CID
        exams_taken = []

        for col_idx, exam_name in enumerate(exams, start=9):  # Column J = index 9
            if str(row[col_idx]).strip().lower() == 'x' or str(row[col_idx]).strip().lower() == 'a'  or str(row[col_idx]).strip().lower() == 'b' :  # Check for 'x' or 'a' or 'b' to indicate they take this course (case-insensitive)
                exams_taken.append(exam_name)

        student_exams[cid] = exams_taken

        # Get the range of rows containing student data (from row 3 onward)




Student exams dictionary:{student_exams}


To create a list of core modules I will read it off the excel from last year 

In [24]:
Core_modules = ["MECH70001 Nuclear Thermal Hydraulics","MECH60004/MECH70042 Introduction to Nuclear Energy A/B","MECH70002 Nuclear Reactor Physics","MECH70008 Mechanical Transmissions Technology","MECH70006 Metal Processing Technology","MECH70021Aircraft Engine Technology","MECH70003 Future Clean Transport Technology","MECH60015 / MECH70030 Professional Engineering Skills (ME3/AME) (ECE exam)"]

Creating a dictionary of fixed modules and their dates, in the format day, slot with day being the day from the first monday (including w/e) of exam season and slot being either morning or afternoon

In [25]:
Fixed_modules = {"BUSI60039 Business Strategy" :[1,1],"BUSI60046 Project Management":[2,1],"ME-ELEC70098 Optimisation":[3,0],"MECH70001 Nuclear Thermal Hydraulics":[3,0],"BUSI60040/BUSI60043 Corporate Finance Online/Finance & Financial Management":[3,1],"MECH60004/MECH70042 Introduction to Nuclear Energy A/B":[4,0],"ME-ELEC70022 Modelling and Control of Multi-body Mechanical Systems":[4,0],"MATE97022 Nuclear Materials 1":[4,0],"ME-MATE70029 Nuclear Fusion":[9,0],"MECH70002 Nuclear Reactor Physics":[10,0],"ME-ELEC70076 Sustainable Electrical Systems":[10,0],"ME ELEC70066 Applied Advanced Optimisation":[10,0],"MECH70020 Combustion, Safety and Fire Dynamics":[11,0],"BIOE70016 Human Neuromechanical Control and Learning":[11,0],"CENG60013 Nuclear Chemical Engineering":[11,0],"MECH70008 Mechanical Transmissions Technology":[17,1],"MECH70006 Metal Processing Technology":[17,1],"MECH70021Aircraft Engine Technology":[17,1],"MECH70003 Future Clean Transport Technology":[17,1],"MECH60015 / MECH70030 Professional Engineering Skills (ME3/AME) (ECE exam)":[18,1]}

Creat a dictionary of each module leader and there respective exams, this needs to use the names used in the students spreadsheet and as such we combine course code and name and find the closest match. Some are disregarded as they're not examinable courses.

In [47]:

# Extract official module names from row 0, columns J onwards (i.e., column 9 onward, 0-indexed)
standardized_names = exams


# Prepare module-leader dictionary
leader_courses = defaultdict(list)
exam_types = dict()
# Loop through rows in the module list
for _, row in leaders_df.iterrows():

    leaders = []
    if pd.notna(row['Module Leader (lecturer 1)']):
        leaders.append(row['Module Leader (lecturer 1)'])

    if pd.notna(row['(UGO Internal) 2nd Exam Marker']):
        leaders.append(row['(UGO Internal) 2nd Exam Marker'])

    name = row['Module Name']
    code = row['Banner Code (New CR)']   # module leader

    # Skip if any required field is missing
    if pd.isna(code) or pd.isna(name) :
        continue

    if len(leaders) == 0 :
        continue

    # Combine code and name
    combined_name = f"{code} {name}"

    # Fuzzy match to standardized names
    best_match, score, _ = process.extractOne(
        combined_name, standardized_names, scorer=fuzz.token_sort_ratio
    )

    if score >= 70: 
        exam_types[best_match] = row['(UGO Internal) Exam Style'] if pd.notna(row['(UGO Internal) Exam Style']) else None
        for leader in leaders:

            if best_match not in leader_courses[leader]:
                leader_courses[leader].append(best_match)
            else:
                a=1
                # print(f"⚠️ Duplicate match skipped for '{combined_name}': '{best_match}' is already listed for {leader}.")
    else:
        a=1
        # print(f"⚠️ Low confidence match for '{combined_name}' (best: '{best_match}', score: {score}).")


# Convert to normal dict if desired
leader_courses = dict(leader_courses)

for exam in exams:
    if exam not in exam_types:
        exam_types[exam] = "Standard"


Make a list of everyone with special exam arrangements

In [27]:
AEA = students_df[students_df.iloc[:, 3].astype(str).str.strip() != ""].iloc[:, 0].tolist()

Make a list of students with 25% extra time to ensure they dont have more than on exam a day

In [28]:
extra_time_students_25 = students_df[students_df.iloc[:, 3].astype(str).str.startswith(("15min/hour", "25% extra time"))].iloc[:, 0].tolist()

print(extra_time_students_25)

[2000005, 2000008, 2000019, 2000026, 2000035, 2000041, 2000042, 2000058, 2000065, 2000068, 2000069, 2000075, 2000083, 2000092, 2000098, 2000099, 2000112, 2000114, 2000116, 2000118, 2000120, 2000121, 2000124, 2000155, 2000156, 2000164, 2000171, 2000175, 2000183, 2000186, 2000191, 2000193, 2000197, 2000199, 2000206, 2000208, 2000223, 2000248, 2000271, 2000277, 2000287, 2000291, 2000296, 2000299, 2000312, 2000317, 2000326, 2000327, 2000335, 2000339, 2000345, 2000353, 2000361, 2000375, 2000390, 2000391, 2000398, 2000403, 2000410]


In [29]:
extra_time_students_50 = students_df[students_df.iloc[:, 3].astype(str).str.startswith(("30min/hour", "50% extra time"))].iloc[:, 0].tolist()

print(extra_time_students_50)

[2000089]


Need to read the bank holidays and forbidden time slots these will have the form [day,slot] the weekends, (days 5, 6 and 12, 13) can be assumed to not have exams and the no exam on morning of the last friday

In [30]:
no_exam_dates = [[5,0],[5,1],[6,0],[6,1],[12,0],[12,1],[13,0],[13,1],[20,0],]

To find the bank holidays we will use the useful dates spreadsheet as this has the start of the summer term and also the bank holidays, first we find the date of the start of summer term, this is made more difficult due to the fact that it is written in the form of day, month to day month year and we need day month year of start. Once this is found the first monday is assumed to be the following monday from this. Then the dates of the bank holidays are found and anything within 20 days is found and appended to the no exam dates

In [31]:


# Initialize
bank_holidays = []


# --- Step 1: Extract bank holidays from col F (names) and G (dates) ---
row = 5
while True:
    name = ws[f"F{row}"].value
    date_cell = ws[f"G{row}"].value
    if name is None or "Term Dates" in str(name):
        break
    if isinstance(date_cell, datetime):
        bank_holidays.append((str(name).strip(), date_cell.date()))
    row += 1

# --- Step 2: Find Summer Term start date from section below ---
summer_start = None
while row < ws.max_row:
    cell_value = ws[f"F{row}"].value
    if cell_value and "Summer Term" in str(cell_value):
        term_range = ws[f"F{row + 1}"].value
        if term_range:
            try:
                # Extract left side of range before "to", drop weekday (e.g., "Fri"), and append year
                start_part = term_range.split("to")[0].strip()
                start_str = re.sub(r"^\w+\s+", "", start_part)  # Removes "Fri", leaves "24 Apr"
                # Try extracting year from second part if present
                year_match = re.search(r"\b\d{4}\b", term_range)
                if year_match:
                    start_str += f" {year_match.group(0)}"
                else:
                    raise ValueError("Year not found in date range.")
                summer_start = parse(start_str, dayfirst=True).date()
            except Exception as e:
                raise ValueError(f"Could not parse Summer Term start: {term_range}") from e
        else:
            raise ValueError("Summer Term range cell is empty.")
        break
    row += 1

if not summer_start:
    raise ValueError("Summer Term start date not found.")

# --- Step 3: Find first Monday on or after summer_start ---
first_monday = summer_start
while first_monday.weekday() != 0:  # 0 = Monday
    first_monday += timedelta(days=1)

# --- Step 4: Find bank holidays within 3 weeks after first Monday ---
for name, bh_date in bank_holidays:
    delta = (bh_date - first_monday).days
    if 0 <= delta <= 20:
        print(f"{name} is {delta} days after first Monday ({bh_date})")
        no_exam_dates.append([delta, 0])
        no_exam_dates.append([delta, 1])

# --- Final Output ---
print("No exam dates:", no_exam_dates)


Early May Bank Holiday is 7 days after first Monday (2026-05-04)
No exam dates: [[5, 0], [5, 1], [6, 0], [6, 1], [12, 0], [12, 1], [13, 0], [13, 1], [20, 0], [7, 0], [7, 1]]


Create a dictionary of rooms using the format ' Room number : [list of uses],[capacity]'

In [32]:
rooms = {
    'CAGB 203': [["Computer", "SEQ",], 65],
    'CAGB 309': [["SEQ"], 54],
    'CAGB 659-652': [["SEQ"], 75],
    'CAGB 747-748': [["SEQ"], 36],
    'CAGB 749-752': [["SEQ"], 75],
    'CAGB 761': [["Computer", "SEQ"], 25],
    'CAGB 762': [["Computer", "SEQ"], 25],
    'SKEM 208': [["Computer", "SEQ"], 35],
    'SKEM 317': [["Computer", "SEQ"], 20],
    'CAGB 320-321': [["AEA"], 10],
    'CAGB 305': [["AEA"], 4],
    'CAGB 349': [["AEA"], 2],
    'CAGB 311': [["AEA"], 1],
    'CAGB 765': [["AEA"],["Computer"], 10],
    'CAGB 527': [["AEA"], 2]
}


Add a dictionary of exams and how many students they have, with SEQ and AEA students

In [33]:
exam_counts = defaultdict(lambda: [0, 0])

for cid, exams_taken in student_exams.items():
    if cid in AEA:
        for exam in exams_taken:
            exam_counts[exam][0] += 1
    else:
        for exam in exams_taken:
            exam_counts[exam][1] += 1